<a href="https://colab.research.google.com/github/dave502/NLP/blob/main/lesson_01/nlp_hw_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [232]:
import re
import pandas as pd
from google.colab import drive
pd.set_option('display.max_colwidth', 140)

In [233]:
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [234]:
path_train = '/drive/My Drive/MLData/train_tweets.csv'
path_test = '/drive/My Drive/MLData/test_tweets.csv'

In [235]:
df = pd.concat([pd.read_csv(f) for f in [path_train, path_test]])

In [236]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49159 entries, 0 to 17196
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      49159 non-null  int64  
 1   label   31962 non-null  float64
 2   tweet   49159 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.5+ MB


**1. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: <br>
● для того, чтобы найти все вхождения паттерна в тексте, необходимо
использовать re.findall(pattern, input_txt) <br>
● для для замены @user на пробел, необходимо использовать re.sub( Текст, выделенный полужирным шрифтом**

In [237]:
# подсчет количества строк с '@user'
sum(df['tweet'].str.findall(r'@[\w]*').str.len())

28075

In [238]:
# замена '@user' на пробел
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'@[\w]*', ' ', x))

In [239]:
# подсчет количества строк с '@user'
sum(df['tweet'].str.findall(r'@[\w]*').str.len())

0

**2. Изменим регистр твитов на нижний с помощью .lower().**

In [240]:
df['tweet'] = df['tweet'].str.lower()

 **3. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя
apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в
тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в
качестве ключа (сокращенного слова), то заменить ключ на значение (полную
версию слова)**

In [241]:
#@title ######Словари apostrophe_dict, short_word_dict, emoticon_dict

apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}



short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}


emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [242]:
def replace_by_dict(text:str, words_dict:dict) -> str:
  """функция замены слов в тексте по словарю"""
  split_text = text.split()
  for i, word in enumerate(split_text):
    split_text[i] = words_dict.get(word) if words_dict.get(word) else word
  return ' '.join(split_text)

In [243]:
# подсчет количества апострофов
sum(df['tweet'].str.findall(r'\w+\'\w+').str.len())

12031

In [244]:
# замена апострофов по словарю
df['tweet'] = df['tweet'].apply(replace_by_dict, args=(apostrophe_dict,))

In [245]:
# подсчет количества апострофов
sum(df['tweet'].str.findall(r'\w+\'\w+').str.len())

2759

*т.е. не все слова с апострофами присутствовали в словаре*

**4. Заменим сокращения на их полные формы, используя short_word_dict. Для этого
воспользуемся функцией, используемой в предыдущем пункте.**

In [246]:
# замена сокращений по словарю
df['tweet'] = df['tweet'].apply(replace_by_dict, args=(short_word_dict,))

**5. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.**

In [247]:
# замена эмоций по словарю
df['tweet'] = df['tweet'].apply(replace_by_dict, args=(emoticon_dict,))

**6 Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'**

In [248]:
# выведем некоторые строки с пунктуацией
indexes = df[df['tweet'].str.findall(r'[^a-zA-Z0-9]').str.len()>0].index[:5]
df['tweet'].iloc[indexes]

0                    when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1    thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked
2                                                                                              bihday your majesty
3                           #model i love you take with you all the time in urð±!!! ðððð ð¦ð¦ð¦
4                                                                              factsguide: society now #motivation
Name: tweet, dtype: object

In [249]:
# замена пунктуации на пробелы
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))

In [250]:
# выведем те же строки
df['tweet'].iloc[indexes]

0                    when a father is dysfunctional and is so selfish he drags his kids into his dysfunction   run
1    thanks for  lyft credit i cannot use cause they do not offer wheelchair vans in pdx   disapointed  getthanked
2                                                                                              bihday your majesty
3                            model i love you take with you all the time in urð       ð   ð   ð   ð   ð   ð   ð   
4                                                                              factsguide  society now  motivation
Name: tweet, dtype: object

**7) Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.**

In [251]:
# вывод нескольких строк со спецсимволами
indexes = df[df['tweet'].str.findall(r'[^a-zA-Z0-9 ]').str.len()>0].index[:5]
df['tweet'].iloc[indexes]

3                                                           model i love you take with you all the time in urð       ð   ð   ð   ð   ð   ð   ð   
6                                                                                                                       camping tomorrow dannyâ  
7     the next school year is the year for exams ð    cannot think about that ð     school  exams  hate  imagine  actorslife  revolutionschool...
8                                                           we won    love the land     allin  cavs  champions  cleveland  clevelandcavaliers â  
10                                   â    ireland consumer price index  mom  climbed from previous 0 2  to 0 5  in may  blog  silver  gold  forex
Name: tweet, dtype: object

In [252]:
# замена спецсимволов на пробелы
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', ' ', x))

In [253]:
# результат
df['tweet'].iloc[indexes]

3                                                           model i love you take with you all the time in ur                                    
6                                                                                                                       camping tomorrow danny   
7     the next school year is the year for exams      cannot think about that       school  exams  hate  imagine  actorslife  revolutionschool...
8                                                           we won    love the land     allin  cavs  champions  cleveland  clevelandcavaliers    
10                                        ireland consumer price index  mom  climbed from previous 0 2  to 0 5  in may  blog  silver  gold  forex
Name: tweet, dtype: object

**8) Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]**'.

In [256]:
# вывод нескольких строк с числами
indexes = df[df['tweet'].str.findall(r'[^a-zA-Z]').str.join('').str.strip().str.len()>0].index[:5]
df['tweet'].iloc[indexes]

5      2 2  huge fan fare and big talking before they leave  chaos and pay disputes when they get there   allshowandnogo
9                                                                           welcome here   I am it has   it is so  gr8  
10               ireland consumer price index  mom  climbed from previous 0 2  to 0 5  in may  blog  silver  gold  forex
12                                                                     i get to see my daddy today    80days  gettingfed
15                                                                 ouch   junior is angry     got7  junior  yugyoem  omg
Name: tweet, dtype: object

In [257]:
# замена чисел на пробелы
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))

In [258]:
# результат
df['tweet'].iloc[indexes]

5           huge fan fare and big talking before they leave  chaos and pay disputes when they get there   allshowandnogo
9                                                                           welcome here   I am it has   it is so  gr   
10               ireland consumer price index  mom  climbed from previous      to      in may  blog  silver  gold  forex
12                                                                     i get to see my daddy today      days  gettingfed
15                                                                 ouch   junior is angry     got   junior  yugyoem  omg
Name: tweet, dtype: object

**9) Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).**

In [259]:
# вывод нескольких строк
indexes = df[df['tweet'].str.findall(r'\s.\s').str.len()>0].index[:3]
df['tweet'].iloc[indexes]

0                    when a father is dysfunctional and is so selfish he drags his kids into his dysfunction   run
1    thanks for  lyft credit i cannot use cause they do not offer wheelchair vans in pdx   disapointed  getthanked
3                            model i love you take with you all the time in ur                                    
Name: tweet, dtype: object

In [260]:
# удаление слов меньше 2-х букв
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))

In [261]:
df['tweet'].iloc[indexes]

0                  when father is dysfunctional and is so selfish he drags his kids into his dysfunction run
1    thanks for lyft credit cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked
3                                                            model love you take with you all the time in ur
Name: tweet, dtype: object

**10) Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.**

In [262]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [263]:
df['tweet_token'] = df['tweet'].apply(lambda x: tknz.word_tokenize(x))

In [266]:
df.head()

,id,label,tweet,tweet_token
0,1,0.0,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]"
1,2,0.0,thanks for lyft credit cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked,"[thanks, for, lyft, credit, can, not, use, cause, they, do, not, offer, wheelchair, vans, in, pdx, disapointed, getthanked]"
2,3,0.0,bihday your majesty,"[bihday, your, majesty]"
3,4,0.0,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, time, in, ur]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]"


**11) Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.**

In [267]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [268]:
stopwords_list= stopwords.words()
df['tweet_token_filtered'] = df['tweet_token'].apply(lambda text: [word for word in text if not word in stopwords_list])

In [269]:
df[['tweet_token', 'tweet_token_filtered']].head()

,tweet_token,tweet_token_filtered
0,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]","[father, dysfunctional, selfish, drags, kids, dysfunction, run]"
1,"[thanks, for, lyft, credit, can, not, use, cause, they, do, not, offer, wheelchair, vans, in, pdx, disapointed, getthanked]","[lyft, credit, offer, wheelchair, vans, pdx, disapointed, getthanked]"
2,"[bihday, your, majesty]","[bihday, majesty]"
3,"[model, love, you, take, with, you, all, the, time, in, ur]","[model, love, time, ur]"
4,"[factsguide, society, now, motivation]","[factsguide, society, motivation]"


**12) Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.**

In [270]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [271]:
df['tweet_stemmed'] = df['tweet_token_filtered'].apply(lambda text: [stemmer.stem(word) for word in text])

In [272]:
df[['tweet_token_filtered', 'tweet_stemmed']].head()

,tweet_token_filtered,tweet_stemmed
0,"[father, dysfunctional, selfish, drags, kids, dysfunction, run]","[father, dysfunct, selfish, drag, kid, dysfunct, run]"
1,"[lyft, credit, offer, wheelchair, vans, pdx, disapointed, getthanked]","[lyft, credit, offer, wheelchair, van, pdx, disapoint, getthank]"
2,"[bihday, majesty]","[bihday, majesti]"
3,"[model, love, time, ur]","[model, love, time, ur]"
4,"[factsguide, society, motivation]","[factsguid, societi, motiv]"


**13) Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.**

In [273]:
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [274]:
df['tweet_lemmatized'] = df['tweet_token_filtered'].apply(lambda text: [lemmatizer.lemmatize(word) for word in text])

In [275]:
df[['tweet_token_filtered', 'tweet_stemmed', 'tweet_lemmatized']].head()

,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,"[father, dysfunctional, selfish, drags, kids, dysfunction, run]","[father, dysfunct, selfish, drag, kid, dysfunct, run]","[father, dysfunctional, selfish, drag, kid, dysfunction, run]"
1,"[lyft, credit, offer, wheelchair, vans, pdx, disapointed, getthanked]","[lyft, credit, offer, wheelchair, van, pdx, disapoint, getthank]","[lyft, credit, offer, wheelchair, van, pdx, disapointed, getthanked]"
2,"[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,"[model, love, time, ur]","[model, love, time, ur]","[model, love, time, ur]"
4,"[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


**14) Сохраним результат предобработки в pickle-файл.**

In [276]:
df.to_pickle('/drive/My Drive/MLData/df_tweets.pkl')